<a href="https://colab.research.google.com/github/megazhuk/pycopy-build-in-colab/blob/main/pycopy_compile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
git_pycopy = "/content/git_pycopy"
git_espidf = "/content/git_espidf"
dir_esp32 = git_pycopy + "/ports/esp32"
dir_mpy_cross = git_pycopy + '/mpy-cross'

In [2]:
!git clone https://github.com/pfalcon/pycopy.git $git_pycopy

Cloning into '/content/git_pycopy'...
remote: Enumerating objects: 5678, done.
remote: Counting objects: 100% (5678/5678), done.
remote: Compressing objects: 100% (2459/2459), done.
remote: Total 127103 (delta 3634), reused 5102 (delta 3203), pack-reused 121425
Receiving objects: 100% (127103/127103), 45.61 MiB | 21.25 MiB/s, done.
Resolving deltas: 100% (92476/92476), done.


In [3]:
fnd_hash_by = 'ESPIDF_SUPHASH_V4'
separator = ' := '
with open(dir_esp32+'/Makefile', 'r') as f:
  while(f):
    tmp_str = f.readline()
    if len(tmp_str) == 0 : break
    # print (tmp_str)
    if fnd_hash_by in tmp_str:
      esp_sha = tmp_str.split(sep=separator, maxsplit=1)[-1][:-1]
      if len(esp_sha)== 40:
        print(f'Found: {esp_sha}')
        break

ESPIDF_SUPHASH_V4 := 4c81978a3e2220674a432a588292a4c860eef27b

Found: 4c81978a3e2220674a432a588292a4c860eef27b


In [4]:
  # Or any path you like.
! git clone https://github.com/espressif/esp-idf.git $git_espidf


Cloning into '/content/git_espidf'...
remote: Enumerating objects: 212437, done.
remote: Total 212437 (delta 0), reused 0 (delta 0), pack-reused 212437
Receiving objects: 100% (212437/212437), 121.46 MiB | 24.08 MiB/s, done.
Resolving deltas: 100% (157298/157298), done.
Checking out files: 100% (7947/7947), done.


In [5]:
print (f'Apply HASH: {esp_sha} to master ESP-IDF repo...')
! cd $git_espidf && git reset --hard $esp_sha

Apply HASH: 4c81978a3e2220674a432a588292a4c860eef27b to master ESP-IDF repo...
HEAD is now at 4c81978a3 Merge branch 'bugfix/doc_version_links_v4.0' into 'release/v4.0'


In [ ]:
print (f'Get submodules...')
! cd $git_espidf && git submodule update --init --recursive

In [ ]:

! cd $git_espidf && ./install.sh

venv_path = '/root/.local/bin'


In [39]:
! cd $git_espidf && . export.sh

Adding ESP-IDF tools to PATH...
Checking if Python packages are up to date...
Python requirements from /content/git_espidf/requirements.txt are satisfied.
Added the following directories to PATH:
  /content/git_espidf/components/esptool_py/esptool
  /content/git_espidf/components/espcoredump
  /content/git_espidf/components/partition_table/
  /root/.espressif/tools/xtensa-esp32-elf/esp-2019r2-8.2.0/xtensa-esp32-elf/bin
  /root/.espressif/tools/esp32ulp-elf/2.28.51.20170517/esp32ulp-elf-binutils/bin
  /root/.espressif/tools/openocd-esp32/v0.10.0-esp32-20190313/openocd-esp32/bin
  /root/.espressif/python_env/idf4.0_py3.6_env/bin
  /content/git_espidf/tools
Done! You can now compile ESP-IDF projects.
Go to the project directory and run:

  idf.py build



In [8]:
!echo RUN: $dir_esp32/source $git_espidf/export.sh

! cd $dir_esp32 && source $git_espidf/export.sh

RUN: /content/git_pycopy/ports/esp32/source /content/git_espidf/export.sh
Adding ESP-IDF tools to PATH...
Checking if Python packages are up to date...
Python requirements from /content/git_espidf/requirements.txt are satisfied.
Added the following directories to PATH:
  /content/git_espidf/components/esptool_py/esptool
  /content/git_espidf/components/espcoredump
  /content/git_espidf/components/partition_table/
  /root/.espressif/tools/xtensa-esp32-elf/esp-2019r2-8.2.0/xtensa-esp32-elf/bin
  /root/.espressif/tools/esp32ulp-elf/2.28.51.20170517/esp32ulp-elf-binutils/bin
  /root/.espressif/tools/openocd-esp32/v0.10.0-esp32-20190313/openocd-esp32/bin
  /root/.espressif/python_env/idf4.0_py3.6_env/bin
  /content/git_espidf/tools
Done! You can now compile ESP-IDF projects.
Go to the project directory and run:

  idf.py build



In [47]:
# export PATH=$PATH:$HOME/esp/crosstool-NG/builds/xtensa-esp32-elf/bin

In [ ]:
print(f'Compile main MPY-CROSS in {dir_mpy_cross}')
! cd $dir_mpy_cross && make V=1

In [14]:

!source $git_pycopy/tools/ci.sh && ci_esp32_idf4_setup && ci_esp32_idf4_path

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 81.5M  100 81.5M    0     0  9950k      0  0:00:08  0:00:08 --:--:-- 12.8M
fatal: destination path 'esp-idf' already exists and is not an empty directory.


In [17]:
dir_mbed = git_espidf + '/components/mbedtls/'
print(f'Patch MBED component in {dir_esp32}')

! cd $dir_mbed && rm -rf mbedtls && git clone -b mbedtls-2.16.5-idf-pycopy https://github.com/pfalcon/mbedtls/

Patch MBED component in /content/git_pycopy/ports/esp32
Cloning into 'mbedtls'...
remote: Enumerating objects: 77358, done.
remote: Total 77358 (delta 0), reused 0 (delta 0), pack-reused 77358
Receiving objects: 100% (77358/77358), 35.86 MiB | 15.34 MiB/s, done.
Resolving deltas: 100% (59716/59716), done.


In [18]:
print(f'Compile esp32 Pycopy in {dir_esp32}')
# ! export ESPIDF=$ESPIDF && cd $dir_esp32 && make submodules && make

! cd $dir_esp32 && source $git_espidf/export.sh && cd $dir_esp32 && make submodules && make

# espidf = "/content/esp-idf"
# ! cd $dir_esp32 && source $espidf/export.sh && cd $dir_esp32 && make submodules && make

Compile esp32 Pycopy in /content/git_pycopy/ports/esp32
Adding ESP-IDF tools to PATH...
Checking if Python packages are up to date...
Python requirements from /content/git_espidf/requirements.txt are satisfied.
Added the following directories to PATH:
  /content/git_espidf/components/esptool_py/esptool
  /content/git_espidf/components/espcoredump
  /content/git_espidf/components/partition_table/
  /root/.espressif/tools/xtensa-esp32-elf/esp-2019r2-8.2.0/xtensa-esp32-elf/bin
  /root/.espressif/tools/esp32ulp-elf/2.28.51.20170517/esp32ulp-elf-binutils/bin
  /root/.espressif/tools/openocd-esp32/v0.10.0-esp32-20190313/openocd-esp32/bin
  /root/.espressif/python_env/idf4.0_py3.6_env/bin
  /content/git_espidf/tools
Done! You can now compile ESP-IDF projects.
Go to the project directory and run:

  idf.py build

Use make V=1 or set BUILD_VERBOSE in your environment to increase build verbosity.
Building with ESP IDF v4
Updating submodules: lib/berkeley-db-1.xx
Use make V=1 or set BUILD_VERBOSE